## **Interpretabilidad de los modelos de machine learning**

### Feature importance

**¿Qué es?**

Serie de técnicas que asignan puntuaciones a las variables independientes de un modelo predictivo en función de su importancia relativa al realizar una predicción sobre la variable dependiente o target. 

IMPORTANTE: Antes de obtener el feature importance siempre hay que evaluar la capacidad predictiva del modelo implementado.

**Tipos**

1. Métodos "built in" en modelos intrinsecamente intepretables (ya vistos al estudiar los modelos lineales y los modelos basados en árboles de decision). 

2. Permutation importance

3. Drop columns importance

4. Shap values


Para todos los ejemplos utilizaremos el dataset de Boston de Skalearn, que plantea un problema de regresión. La descripción de las variables de entrada se puede encontrar [aquí](https://scikit-learn.org/stable/datasets/toy_dataset.html#boston-dataset)

La métrica que utilizaremos es el .score() de los modelos de regresión, es decire, el [R2 score](https://scikit-learn.org/stable/modules/model_evaluation.html#r2-score).

In [ ]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
import seaborn as sns
from sklearn import metrics
from sklearn.datasets import load_boston
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor

In [ ]:
#Importamos el Boston dataset de Skalearn
boston = load_boston()
print(boston.DESCR)

In [ ]:
#Construimos el dataframe para echarle un vistazo

boston_df = pd.DataFrame(boston.data,                      
                         columns=boston.feature_names)
boston_df["MEDV"]= boston.target

boston_df.head()

In [ ]:
#Vemos que aspecto tiene la relación entre las variables 
#con una matriz de correlación

plt.figure(figsize=(10,10))

sns.heatmap(boston_df.corr(),
            vmin=-1,
            vmax=1,
            center=0,
            cmap=sns.diverging_palette(220, 20, as_cmap=True),
            square=True,
            annot=True,
            linewidths=.5);

In [ ]:
#Espliteamos para montar el modelo

X = boston_df.drop("MEDV", axis = 1)
y = boston_df["MEDV"]
X_train, X_test, y_train, y_test = train_test_split(X, 
                                                    y, 
                                                    test_size=0.20, 
                                                    random_state=42)

#### Random Forest features importances 

In [ ]:
#Implementamos un random forest

from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import r2_score

rnd_reg = RandomForestRegressor(n_estimators = 200, 
                                max_depth = 4,
                                random_state = 42)
rnd_reg.fit(X_train, y_train)

print("R2 score train",rnd_reg.score(X_train,y_train))
print("R2 score test",rnd_reg.score(X_test,y_test))


In [ ]:
#Obtenemos el feature_importances
rnd_reg.feature_importances_

In [ ]:
fi_rnd= pd.DataFrame(rnd_reg.feature_importances_,
                          boston.feature_names, 
                          columns = ["Feature imp. RND"]).sort_values("Feature imp. RND", ascending=False)
fi_rnd

In [ ]:
#Visualizamos
fi_rnd.sort_values("Feature imp. RND").plot.barh(y='Feature imp. RND');

####  Permutation importance

In [ ]:
#Implementamos el permutation importance

from sklearn.inspection import permutation_importance

perm_train = permutation_importance(estimator=rnd_reg,
                                    X = X_train,
                                    y = y_train,
                                    n_repeats = 10,
                                    random_state=42,
                                    scoring="r2")

#n_repeats = Number of times to permute a feature.

Obtenemos el permutation importance para cada variable, como la media del impacto de las permutaciones de sus valores sobre el rendimiento del modelo (r2) a lo largo de las repeticiones efectuadas.

In [ ]:
#Raw permutation importance scores (13 features * 10 permutaciones)
perm_train.importances.size
### CODE ####

In [ ]:
# Mean of feature importance over n_repeats.
perm_train.importances_mean
### CODE ####

In [ ]:
#Montamos el dataframe
df_perm_train = pd.DataFrame(perm_train.importances_mean,
                          boston.feature_names, 
                          columns = ["Feature imp. RND"]).sort_values("Feature imp. RND", ascending=False)
### CODE ####
df_perm_train

In [ ]:
### CODE ####
#Visualizamos
df_perm_train.sort_values("Feature imp. RND").plot.barh(y='Feature imp. RND');

In [ ]:
#Obtenemos el permutation importance sobre los datos de test
perm_test = permutation_importance(estimator=rnd_reg,
                                    X = X_test,
                                    y = y_test,
                                    n_repeats = 10,
                                    random_state=42,
                                    scoring="r2")


In [ ]:
# Lo pasamos a un dataframe
df_perm_test = pd.DataFrame(perm_test.importances_mean,
                          boston.feature_names, 
                          columns = ["Feature imp. RND"]).sort_values("Feature imp. RND", ascending=False)
### CODE ####
df_perm_test

In [ ]:
#Graficamos
df_perm_test.sort_values("Feature imp. RND").plot.barh(y='Feature imp. RND');

### Drop columns

In [ ]:
# Primero vemos como se haría manualmente

from sklearn.base import clone 

# Create un unfitted model with the exact same specification as the one initially trained
model_clone = clone(rnd_reg)
# set random_state for comparability
model_clone.random_state = 42
# training and scoring the benchmark model
model_clone.fit(X_train, y_train)
baseline_score = model_clone.score(X_train, y_train)
# list for storing feature importances
importances = []
    
# iterating over all columns and storing feature importance (difference between benchmark and new model)
for col in X_train.columns:
    model_clone = clone(rnd_reg)
    model_clone.random_state = 42
    model_clone.fit(X_train.drop(col, axis = 1), y_train)
    drop_col_score = model_clone.score(X_train.drop(col, axis = 1), y_train)
    importances.append(baseline_score - drop_col_score)

In [ ]:
drop_col_manual = pd.DataFrame(importances, 
                              X_train.columns, 
                              columns = ["Drop columns importance"]).sort_values("Drop columns importance", ascending = False)
drop_col_manual

In [ ]:
#Graficamos
drop_col_manual.plot.barh(y= "Drop columns importance");

#### SHAP

In [ ]:
import shap

In [ ]:
explainer = shap.TreeExplainer(rnd_reg)
shap_values = explainer.shap_values(X)

In [ ]:
shap.summary_plot(shap_values, X)

In [ ]:
shap.summary_plot(shap_values, X, plot_type='bar')